In [12]:
import tensorflow as tf
import os 
import pandas as pd 
import numpy as np
from tensorflow.keras import layers

In [11]:
print(tf.config.list_physical_devices('GPU'))
# os.environ['XLA_FLAGS']
# os.environ是一个字典，可以通过get方法获得对应的值
os.environ['SYSTEMROOT']
os.environ['TEMP']
# 设置系统环境变量
# os.putenv('环境变量名称','环境变量值') 
# 修改环境变量
# os.environ['环境变量名称']='新环境变量名称'

[PhysicalDevice(name='/physical_device:GPU:0', device_type='GPU')]


'C:\\WINDOWS\\Temp'

In [13]:
abalone_train = pd.read_csv(
    "https://storage.googleapis.com/download.tensorflow.org/data/abalone_train.csv",
    names=["Length", "Diameter", "Height", "Whole weight", "Shucked weight",
           "Viscera weight", "Shell weight", "Age"])

abalone_train.head()

,Length,Diameter,Height,Whole weight,Shucked weight,Viscera weight,Shell weight,Age
0,0.435,0.335,0.110,0.334,0.1355,0.0775,0.0965,7
1,0.585,0.450,0.125,0.874,0.3545,0.2075,0.2250,6
2,0.655,0.510,0.160,1.092,0.3960,0.2825,0.3700,14
3,0.545,0.425,0.125,0.768,0.2940,0.1495,0.2600,16
4,0.545,0.420,0.130,0.879,0.3740,0.1695,0.2300,13


In [15]:
abalone_features =abalone_train.copy()
abalone_labels = abalone_features.pop("Age")
abalone_features = np.array(abalone_features)

array([[0.435 , 0.335 , 0.11  , ..., 0.1355, 0.0775, 0.0965],
       [0.585 , 0.45  , 0.125 , ..., 0.3545, 0.2075, 0.225 ],
       [0.655 , 0.51  , 0.16  , ..., 0.396 , 0.2825, 0.37  ],
       ...,
       [0.53  , 0.42  , 0.13  , ..., 0.3745, 0.167 , 0.249 ],
       [0.395 , 0.315 , 0.105 , ..., 0.1185, 0.091 , 0.1195],
       [0.45  , 0.355 , 0.12  , ..., 0.1145, 0.0665, 0.16  ]])

In [ ]:
normalize = layers.Normalization()
normalize.adapt(abalone_features)

In [ ]:
abalone_model = tf.keras.Sequential([layers.Dense(64), layers.Dense(1)])

## Mixed data types

Download Titanic dataset

In [16]:
titanic = pd.read_csv("https://storage.googleapis.com/tf-datasets/titanic/train.csv")
titanic.head()

,survived,sex,age,n_siblings_spouses,parch,fare,class,deck,embark_town,alone
0,0,male,22.0,1,0,7.2500,Third,unknown,Southampton,n
1,1,female,38.0,1,0,71.2833,First,C,Cherbourg,n
2,1,female,26.0,0,0,7.9250,Third,unknown,Southampton,y
3,1,female,35.0,1,0,53.1000,First,C,Southampton,n
4,0,male,28.0,0,0,8.4583,Third,unknown,Queenstown,y


In [27]:
titanic_X = titanic.copy()
titanic_Y = titanic_X.pop('survived')

The Keras preprocessing layers are parts of the model. Keras functional API operates on 'symbolic' tensors

In [22]:
# create a symbolic input
# input = tf.keras.Input(shape=(784,), dtype=tf.float32)  # TensorShape([None, 784])
input = tf.keras.Input(shape=(), dtype=tf.float32)
result = 2 * input + 1
img_input = tf.keras.Input(shape=(32, 32, 3)) # 这个图像的维度顺序和torch不一样
calc = tf.keras.Model(inputs=input, outputs=result)
print(calc(1).numpy())

3.0


## Match the names and data-types of the CSV columns
build a set of symbolic tf.keras.Input objects, matching the names and data-types of the CSV columns

In [37]:
inputs = {}

for name, col in titanic_X.items():
    dtype = col.dtype
    if dtype == object:
        dtype = tf.string
    else:
        dtype = tf.float32
    inputs[name] = tf.keras.Input(shape=(1,), name=name, dtype=dtype)

## Concatenate the numeric inputs together

In [38]:
numeric_X = {name:input for name,input in inputs.items()
            if input.dtype==tf.float32}
x = layers.Concatenate()(list(numeric_X.values()))
norm = layers.Normalization()
# 在对数据进行处理前，需要调用adapt，对数据进行学习，adapt必须在调用fit，evaluate，predict之前
norm.adapt(np.array(titanic[numeric_X.keys()])) # 调用实际的数据
all_numeric_inputs = norm(x) # 抽象的预处理层
preprocessed_inputs = [all_numeric_inputs]
all_numeric_inputs

<KerasTensor: shape=(None, 4) dtype=float32 (created by layer 'normalization_4')>

## Build preprocessing model
For the string inputs use the `tf.keras.layers.StringLookup` function to map from strings to integer indices in a vocabulary. Next, use `tf.keras.layers.CategoryEncoding` to convert the indexes into float32 data appropriate for the model

In [39]:
for name, input in inputs.items():
    if input.dtype==tf.float32:
        continue
    # 指定vocabulary后可以不用使用adapt()
    lookup = layers.StringLookup(vocabulary=np.unique(titanic_X[name]), output_mode='one_hot')
#     emb = layers.Embedding(input_dim=lookup.vocabulary_size(), output_dim=32)
    
    x = lookup(input)
#     x = emb(x)
    preprocessed_inputs.append(x)

In [45]:
# 拼接到一起
preprocessed_inputs_cat = layers.Concatenate()(preprocessed_inputs)
# 建立model
titanic_preprocessing_model = tf.keras.Model(inputs, preprocessed_inputs_cat)
tf.keras.utils.plot_model(model = titanic_preprocessing_model , rankdir="LR", dpi=72, show_shapes=True)

You must install pydot (`pip install pydot`) and install graphviz (see instructions at https://graphviz.gitlab.io/download/) for plot_model to work.


## Convert pandas DataFrame to Dict

In [46]:
titanic_features_dict = {name: np.array(value)
                        for name, value in titanic_X.items()}


In [47]:
features_dict = {name:values[:1] for name, values in titanic_features_dict.items()}
titanic_preprocessing_model(features_dict)

<tf.Tensor: shape=(1, 28), dtype=float32, numpy=
array([[-0.610415 ,  0.395198 , -0.4790527, -0.4974028,  0.       ,
         0.       ,  1.       ,  0.       ,  0.       ,  0.       ,
         1.       ,  0.       ,  0.       ,  0.       ,  0.       ,
         0.       ,  0.       ,  0.       ,  0.       ,  1.       ,
         0.       ,  0.       ,  0.       ,  1.       ,  0.       ,
         0.       ,  1.       ,  0.       ]], dtype=float32)>

## Build the whole model

In [48]:
def titanic_model(preprocessing_model, inputs):
    """
    :param preprocessing_model: 定义好的预处理模型 
    :param inputs: 输入层，tf.keras.Input的集合
    """
    body = tf.keras.Sequential([
        layers.Dense(64),
        layers.Dense(1)
    ])
    preprocessed_inputs = preprocessing_model(inputs)
    result = body(preprocessed_inputs)
    # 通过inputs和result建立model
    model = tf.keras.Model(inputs, result)
    
    model.compile(loss=tf.keras.losses.BinaryCrossentropy(from_logits=True),
                 optimizer=tf.keras.optimizers.Adam(learning_rate=0.001))
    return model
titanic_model = titanic_model(titanic_preprocessing_model, inputs)

In [49]:
# 注意输入为
titanic_model.fit(x=titanic_features_dict, y=titanic_Y, epochs=50)

Epoch 1/50
20/20 [==============================] - 4s 9ms/step - loss: 0.7506
Epoch 2/50
20/20 [==============================] - 0s 8ms/step - loss: 0.6095
Epoch 3/50
20/20 [==============================] - 0s 8ms/step - loss: 0.5349
Epoch 4/50
20/20 [==============================] - 0s 8ms/step - loss: 0.4936
Epoch 5/50
20/20 [==============================] - 0s 8ms/step - loss: 0.4682
Epoch 6/50
20/20 [==============================] - 0s 8ms/step - loss: 0.4502
Epoch 7/50
20/20 [==============================] - 0s 8ms/step - loss: 0.4391
Epoch 8/50
20/20 [==============================] - 0s 9ms/step - loss: 0.4322
Epoch 9/50
20/20 [==============================] - 0s 10ms/step - loss: 0.4270
Epoch 10/50
20/20 [==============================] - 0s 8ms/step - loss: 0.4244
Epoch 11/50
20/20 [==============================] - 0s 8ms/step - loss: 0.4219
Epoch 12/50
20/20 [==============================] - 0s 9ms/step - loss: 0.4216
Epoch 13/50
20/20 [=============================

In [50]:
titanic_model.save('test')
reloaded = tf.keras.models.load_model("test")

INFO:tensorflow:Assets written to: test\assets
